# Evaluation metrics

## Geometry

### Chamfer distance
A standard metric to measure the shape dissimilarity between point clouds in point cloud completion.
The chamfer distance is defined as the sum of the distances from each point in A to its nearest neighbor in B, plus the sum of the distances from each point in B to its nearest neighbor in A

In [5]:
import open3d as o3d
import numpy as np

def get_chamfer_distance(pcd1,pcd2):
    # Calculate distances of pcd1 to pcd2. 
    dist_pc1_pc2 = pcd1.compute_point_cloud_distance(pcd2)
    dist_pc1_pc2 = np.asarray(dist_pc1_pc2)

    # Calculate distances of pcd2 to pcd1. 
    dist_pc2_pc1 = pcd2.compute_point_cloud_distance(pcd1)
    dist_pc2_pc1 = np.asarray(dist_pc2_pc1)

    chamferDistance = np.sum((dist_pc2_pc1, dist_pc1_pc2))
    return chamferDistance


In [11]:
mesh1 = o3d.io.read_triangle_mesh("../data/sofa/Sofa.obj")
pcd1 = mesh1.sample_points_uniformly(number_of_points=2500)

mesh2 = o3d.io.read_triangle_mesh("../data/sofa/Sofa_completed_0.obj")
pcd2 = mesh2.sample_points_uniformly(number_of_points=2500)

mesh3 = o3d.io.read_triangle_mesh("../data/sofa/Sofa_completed_1.obj")
pcd3 = mesh3.sample_points_uniformly(number_of_points=2500)

get_chamfer_distance(pcd2,pcd3)


195.9641965506257

### IOU

Intersection over union. Using a volumetric comparison of 2 sdfs.

In [61]:
from context import generationtools as gnt
import trimesh

mesh1 = trimesh.load("../data/sofa/Sofa_completed_0.obj")
mesh2 = trimesh.load("../data/sofa/Sofa_completed_2.obj")

sdf1 = gnt.mesh_to_sdf_tensor(mesh1, recenter=False)
sdf2 = gnt.mesh_to_sdf_tensor(mesh2, recenter=False)

In [62]:
print(np.sum(sdf1[0] < 0))
print(np.sum(sdf2[0] < 0))

# union = get the closest value per voxel
unionSdf = np.minimum(sdf1[0],sdf2[0])
unionCount = np.sum(unionSdf < 0)
 
# Intersection = get the longest value per voxel
intersectionSdf = np.maximum(sdf1[0],sdf2[0])
intersectionCount = np.sum(intersectionSdf < 0)

iou = intersectionCount/unionCount

print("IOU: " + str(iou))

combined = trimesh.util.concatenate( [ sdf1[1], sdf2[1] ] )
trimesh.repair.fix_inversion(combined)
combined.show()


111973
95390
IOU: 0.7276650697771297


## Material

### Percentage correctness

Since The material prediction is pointwise, we perform a binary true or false percentage prediction check.

In [42]:
import trimesh
import numpy as np

# todo filter by only the filled in points
mesh1 = trimesh.load("../data/sofa/1c17cc67b8c747c3febad4f49b26ec52_normalized-partial-0_color_reconstruction.obj")
mesh2 = trimesh.load("../data/sofa/1c17cc67b8c747c3febad4f49b26ec52_normalized-partial-1_color_reconstruction.obj")
#print(mesh1.vertices.size)
#print(mesh1.colors>125)
#print(mesh2.colors>125)
count = np.sum(np.all(np.equal(mesh1.colors>125, mesh2.colors>125), axis=1))

print("The percentage of correctly segmented points is: " + str(count / (mesh1.vertices.size/3)))

The percentage of correctly segmented points is: 0.96504


In [43]:
mesh1.show()

## Texture

### Texture similarity

A feature vector is calculated for each patch in the predicted inpainted texture and compared to the ground truth (if it exists). 

In [18]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
efficientnet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
efficientnet.eval().to(device)
print("Model loaded")

Model loaded


Using cache found in C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub
Using cache found in C:\Users\jelle/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


In [22]:
import torchvision.transforms as transforms
from PIL import Image

def load_jpeg_from_file(image, image_size, cuda=True):
    img_transforms = transforms.Compose(
        [
            transforms.Resize(image_size + 32),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
        ]
    )

    img = img_transforms(image)
    with torch.no_grad():
        # mean and std are not multiplied by 255 as they are in training script
        # torch dataloader reads data into bytes whereas loading directly
        # through PIL creates a tensor with floats in [0,1] range
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

        if cuda:
            mean = mean.cuda()
            std = std.cuda()
            img = img.cuda()
        img = img.float()

        input = img.unsqueeze(0).sub_(mean).div_(std)

    return input

In [38]:
import cv2

#image1 = cv2.imread('../data/sofa/1c17cc67b8c747c3febad4f49b26ec52/images/texture2.jpg')
image1 = cv2.imread('../data/sofa/1c6eb96eab5e75b67b79156a61ad4c01/images/texture2.jpg')
image1_rotated = cv2.rotate(image1, cv2.ROTATE_180)
image2 = cv2.imread('../data/sofa/1c6eb96eab5e75b67b79156a61ad4c01/images/texture4.jpg')

imageTensor1 = load_jpeg_from_file(Image.fromarray(image1), image1.shape[0], False)
imageTensor1_rotated = load_jpeg_from_file(Image.fromarray(image1_rotated), image1_rotated.shape[0], False)
imageTensor2 = load_jpeg_from_file(Image.fromarray(image2), image2.shape[0], False)
imageFeatures1 = efficientnet(imageTensor1)
imageFeatures1_rotated = efficientnet(imageTensor1_rotated)
imageFeatures2 = efficientnet(imageTensor2)

print(imageFeatures1.shape)

torch.Size([1, 1000])


In [40]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(imageFeatures1, imageFeatures2)
output_rotated = cos(imageFeatures1, imageFeatures1_rotated)
print("The Patch similarity of a different is: ")
print(output[0].item())
print("The Patch similarity of a similar image is: ")
print(output_rotated[0].item())

The Patch similarity of a different is: 
0.1623810976743698
The Patch similarity of a similar image is: 
0.9866843223571777
